In [1]:
import pandas as pd 
from db import *
from config_g import *
# from config_poc import *


In [2]:
import json

```
select  c.table_name, c.column_name, c.data_type, c.*
from information_schema.columns c
where 1=1
and c.table_catalog = 'faculty-Cornell-CS'
and c.table_schema = 'main'
and c.table_name in (
	--'t_note'
	select table_name from information_schema.tables t
	where t.table_catalog = 'faculty-Cornell-CS'
	and t.table_schema = 'main'
)
order by c.table_catalog,c.table_schema,c.table_name, c.ordinal_position
```

In [20]:
def _gen_label(col):
    "Convert table column into form label"
    if col == 'ts_created': return "Created At"
    if "_" not in col:
        if col.upper() in ["URL","ID"]:
            return col.upper()
        elif col.upper() == "TS":
            return "Timestamp"
        return col.capitalize()

    cols = []
    for c in col.split("_"):
        c  = c.strip()
        if not c: continue
        cols.append(c.capitalize())
    return " ".join(cols)

def _prepare_col_props(selected_tbls):
    col_props = {}
    for tbl in selected_tbls:
        cols_def = {}
        for col in TABLE_COLUMNS[tbl]:
            col_def = {}
            for p in PROPS:
                if "is_" in p:
                    col_def.update({p: False})
                elif p == 'form_column':
                    col_def.update({p: "col1-1"})
                elif p == 'widget_type':
                    col_def.update({p: "text_input"})
                else:
                    col_def.update({p: _gen_label(col)})
            cols_def[col] = col_def
        col_props[tbl] = cols_def

    return col_props

def _get_columns(table_name, prop_name="is_visible"):
    cols_bool = []
    cols_text = {}
    for k,v in COLUMN_PROPS[table_name].items():
        if prop_name.startswith("is_") and v.get(prop_name, False):
            cols_bool.append(k)
            
        if not prop_name.startswith("is_"):
            val = v.get(prop_name, "")
            if val:
                cols_text.update({k: val})
    
    return cols_bool or cols_text

def _parse_column_props():
    """parse COLUMN_PROPS map
    """
    col_defs = {}
    for table_name in COLUMN_PROPS.keys():
        defs = {}
        cols_widget_type = {}
        cols_label_text = {}
        for p in PROPS:
            res = _get_columns(table_name, prop_name=p)
            # print(f"{p}: {res}")
            if p == 'widget_type':
                cols_widget_type = res
            elif p == 'label_text':
                cols_label_text = res
            defs[p] = res
            
        # reset label
        for col in cols_widget_type.keys():
            label = cols_label_text.get(col, "")
            if not label:
                label = _gen_label(col)
            cols_label_text.update({col : label})
        # print(cols_label_text)
        defs['label_text'] = cols_label_text
        defs['all_columns'] = list(cols_widget_type.keys())

        # sort form_column alpha-numerically
        # max number of form columns = 3
        # add them
        tmp = {}
        for i in range(1,4):
            m = {k:v for k,v in defs['form_column'].items() if v.startswith(f"col{i}-")}
            tmp[f"col{str(i)}_columns"] = sorted(m, key=m.__getitem__)        
        defs.update(tmp)
        col_defs[table_name] = defs

        
    return col_defs

In [4]:
sql_stmt = """
select  c.table_name, c.column_name, c.data_type --,  c.*
from information_schema.columns c
where 1=1
--and c.table_catalog like 'faculty-Cornell-CS%'
and c.table_schema = 'main'
and c.table_name in (
	--'t_note'
	select table_name from information_schema.tables t
	where 1=1
	--and t.table_catalog like 'faculty-Cornell-CS%'
	and t.table_schema = 'main'
)
and c.table_name not like 't1_%'
order by c.table_catalog,c.table_schema,c.table_name, c.ordinal_position
;
"""

In [5]:
with DBConn() as _conn:
    df = pd.read_sql(sql_stmt, _conn)

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_9040\496018617.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_stmt, _conn)


In [6]:
df

,table_name,column_name,data_type
0,g_column_props,table_name,VARCHAR
1,g_column_props,col_name,VARCHAR
2,g_column_props,is_system_col,BOOLEAN
3,g_column_props,is_user_key,BOOLEAN
4,g_column_props,is_required,BOOLEAN
...,...,...,...
157,t_work,tags,VARCHAR
158,t_work,note,VARCHAR
159,t_work,ts_created,VARCHAR
160,t_work,id,VARCHAR


In [7]:
df.to_records()

rec.array([(  0, 'g_column_props', 'table_name', 'VARCHAR'),
           (  1, 'g_column_props', 'col_name', 'VARCHAR'),
           (  2, 'g_column_props', 'is_system_col', 'BOOLEAN'),
           (  3, 'g_column_props', 'is_user_key', 'BOOLEAN'),
           (  4, 'g_column_props', 'is_required', 'BOOLEAN'),
           (  5, 'g_column_props', 'is_visible', 'BOOLEAN'),
           (  6, 'g_column_props', 'is_editable', 'BOOLEAN'),
           (  7, 'g_column_props', 'is_clickable', 'BOOLEAN'),
           (  8, 'g_column_props', 'form_column', 'VARCHAR'),
           (  9, 'g_column_props', 'widget_type', 'field_ui_type'),
           ( 10, 'g_column_props', 'label_text', 'VARCHAR'),
           ( 11, 'g_column_props', 'kwargs', 'VARCHAR'),
           ( 12, 'g_entity', 'id', 'VARCHAR'),
           ( 13, 'g_entity', 'uid', 'VARCHAR'),
           ( 14, 'g_entity', 'ts', 'VARCHAR'),
           ( 15, 'g_entity', 'entity_type', 'VARCHAR'),
           ( 16, 'g_entity', 'name', 'VARCHAR'),
           

In [8]:
tbl_map = {}
for idx, tbl, col, typ in df.to_records():
    if tbl not in tbl_map:
        tbl_map[tbl] = [(col,typ)]
    else:
        tmp = tbl_map[tbl]
        tmp.append((col,typ))
        tbl_map[tbl] = tmp
tbl_map

{'g_column_props': [('table_name', 'VARCHAR'),
  ('col_name', 'VARCHAR'),
  ('is_system_col', 'BOOLEAN'),
  ('is_user_key', 'BOOLEAN'),
  ('is_required', 'BOOLEAN'),
  ('is_visible', 'BOOLEAN'),
  ('is_editable', 'BOOLEAN'),
  ('is_clickable', 'BOOLEAN'),
  ('form_column', 'VARCHAR'),
  ('widget_type', 'field_ui_type'),
  ('label_text', 'VARCHAR'),
  ('kwargs', 'VARCHAR')],
 'g_entity': [('id', 'VARCHAR'),
  ('uid', 'VARCHAR'),
  ('ts', 'VARCHAR'),
  ('entity_type', 'VARCHAR'),
  ('name', 'VARCHAR'),
  ('url', 'VARCHAR'),
  ('note', 'VARCHAR')],
 'g_note': [('id', 'VARCHAR'),
  ('uid', 'VARCHAR'),
  ('ts', 'VARCHAR'),
  ('name', 'VARCHAR'),
  ('url', 'VARCHAR'),
  ('note', 'VARCHAR'),
  ('tags', 'VARCHAR'),
  ('ref_key', 'VARCHAR'),
  ('ref_val', 'VARCHAR')],
 'g_person': [('id', 'VARCHAR'),
  ('uid', 'VARCHAR'),
  ('ts', 'VARCHAR'),
  ('person_type', 'VARCHAR'),
  ('name', 'VARCHAR'),
  ('url', 'VARCHAR'),
  ('note', 'VARCHAR'),
  ('email', 'VARCHAR'),
  ('first_name', 'VARCHAR'),
  (

In [9]:
for tbl in tbl_map.keys():
    cols = tbl_map[tbl] 
    tmp = []
    for col in cols:
        tmp.append(f"{col[0]} {col[1]}")
    col_list = "\n\t    ,".join(tmp)
    print(f"""
        create table {tbl.replace("t_", "g_")} (
            {col_list}
        );\n
    """)


        create table g_column_props (
            table_name VARCHAR
	    ,col_name VARCHAR
	    ,is_system_col BOOLEAN
	    ,is_user_key BOOLEAN
	    ,is_required BOOLEAN
	    ,is_visible BOOLEAN
	    ,is_editable BOOLEAN
	    ,is_clickable BOOLEAN
	    ,form_column VARCHAR
	    ,widget_type field_ui_type
	    ,label_text VARCHAR
	    ,kwargs VARCHAR
        );

    

        create table g_entity (
            id VARCHAR
	    ,uid VARCHAR
	    ,ts VARCHAR
	    ,entity_type VARCHAR
	    ,name VARCHAR
	    ,url VARCHAR
	    ,note VARCHAR
        );

    

        create table g_note (
            id VARCHAR
	    ,uid VARCHAR
	    ,ts VARCHAR
	    ,name VARCHAR
	    ,url VARCHAR
	    ,note VARCHAR
	    ,tags VARCHAR
	    ,ref_key VARCHAR
	    ,ref_val VARCHAR
        );

    

        create table g_person (
            id VARCHAR
	    ,uid VARCHAR
	    ,ts VARCHAR
	    ,person_type VARCHAR
	    ,name VARCHAR
	    ,url VARCHAR
	    ,note VARCHAR
	    ,email VARCHAR
	    ,first_name VAR

In [10]:
df.to_csv("tab_cols-2023-04-28.csv", index=False)

In [11]:
df_list= df.groupby('table_name')['column_name', 'data_type'].apply(list)

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_9040\1589112563.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_list= df.groupby('table_name')['column_name', 'data_type'].apply(list)


In [12]:
TABLE_COLUMNS = df_list.to_dict()

In [13]:
TABLE_COLUMNS

{'g_column_props': ['column_name', 'data_type'],
 'g_entity': ['column_name', 'data_type'],
 'g_note': ['column_name', 'data_type'],
 'g_person': ['column_name', 'data_type'],
 'g_relation': ['column_name', 'data_type'],
 'g_work': ['column_name', 'data_type'],
 't_column_props': ['column_name', 'data_type'],
 't_faculty': ['column_name', 'data_type'],
 't_note': ['column_name', 'data_type'],
 't_org': ['column_name', 'data_type'],
 't_person': ['column_name', 'data_type'],
 't_person_team': ['column_name', 'data_type'],
 't_person_work': ['column_name', 'data_type'],
 't_research_group': ['column_name', 'data_type'],
 't_team': ['column_name', 'data_type'],
 't_work': ['column_name', 'data_type']}

In [14]:
PROPS = [c for c in TABLE_COLUMNS['t_column_props'] if c not in ['table_name','col_name']]

In [15]:
PROPS

['column_name', 'data_type']

In [16]:
TABLE_COLUMNS.keys()

dict_keys(['g_column_props', 'g_entity', 'g_note', 'g_person', 'g_relation', 'g_work', 't_column_props', 't_faculty', 't_note', 't_org', 't_person', 't_person_team', 't_person_work', 't_research_group', 't_team', 't_work'])

In [15]:
COLUMN_PROPS.keys()

dict_keys(['t_note', 't_team', 't_work', 't_person_team', 't_person_work', 't_research_group', 't_person'])

In [11]:
selected_tbls = [t for t in TABLE_COLUMNS.keys() if t not in [
            "t_person_team", "t_person_work", "t_team", "t_work", 't_note', 't_column_props']]

In [12]:
selected_tbls = ['t_person', 't_research_group']

In [21]:
col_props = _prepare_col_props(selected_tbls)

In [23]:
with open("col_props-1.json", "w") as f:
    f.write(json.dumps(col_props))

In [23]:
from config_g import *

In [24]:
COLUMN_DEFS = _parse_column_props()

In [25]:
COLUMN_DEFS

{'g_entity': {'is_system_col': ['id'],
  'is_user_key': ['name'],
  'is_required': ['name', 'id'],
  'is_visible': ['name', 'url', 'entity_type', 'id', 'note'],
  'is_editable': ['name', 'url', 'entity_type', 'note'],
  'is_clickable': ['url'],
  'form_column': {'name': 'col1-1',
   'url': 'col1-2',
   'entity_type': 'col1-3',
   'id': 'col2-1',
   'note': 'col2-2'},
  'widget_type': {'name': 'text_input',
   'url': 'text_input',
   'entity_type': 'selectbox',
   'id': 'text_input',
   'note': 'text_area'},
  'label_text': {'name': 'Name',
   'url': 'URL',
   'id': 'ID',
   'note': 'Note',
   'entity_type': 'Entity Type'},
  'kwargs': {},
  'all_columns': ['name', 'url', 'entity_type', 'id', 'note'],
  'col1_columns': ['name', 'url', 'entity_type'],
  'col2_columns': ['id', 'note'],
  'col3_columns': []},
 'g_person': {'is_system_col': ['id'],
  'is_user_key': ['name'],
  'is_required': ['name', 'id'],
  'is_visible': ['name',
   'url',
   'email',
   'job_title',
   'research_area',
 

In [26]:
table_name = "g_person"
COL_DEFS = COLUMN_DEFS[table_name]
visible_columns = COL_DEFS["is_visible"]
system_columns = COL_DEFS["is_system_col"]
form_columns = COL_DEFS["form_column"]
col_labels = COL_DEFS["label_text"]
widget_types = COL_DEFS["widget_type"]

In [27]:
key_columns = COL_DEFS["is_user_key"]
key_columns

['name']

In [1]:
s = "publication, preprint, talk, poster, project, startup, company"
WORK_TYPES = [c.strip() for c in s.split(",") if c.strip()]
WORK_TYPES

['publication', 'preprint', 'talk', 'poster', 'project', 'startup', 'company']

In [19]:
form_cols = {
    "note": "col2-2",
    "url": "col1-2",
    "name": "col1-1",
    "id": "col2-1",
}

res = {}
for i in range(1,4):
    m = {k:v for k,v in form_cols.items() if v.startswith(f"col{i}-")}
    res[f"col{str(i)}_columns"] = sorted(m, key=m.__getitem__)

In [20]:
res

{'col1_columns': ['name', 'url'],
 'col2_columns': ['id', 'note'],
 'col3_columns': []}

In [21]:
res2 = {}

In [22]:
res2.update(res)

In [23]:
res2

{'col1_columns': ['name', 'url'],
 'col2_columns': ['id', 'note'],
 'col3_columns': []}

In [24]:
c = [1,2]
c2 = c.copy()
c2

[1, 2]

In [34]:
def _move_sys_col_end(cols, sys_cols=["id"]):
    """move id, ts sys column to last position
    """
    new_sys_cols = []
    new_cols = []
    for c in cols:
        if c in sys_cols:
            new_sys_cols.append(c)
        else:
            new_cols.append(c)
    if sys_cols:
        if len(new_cols) == len(cols):
            return new_cols
        else:
            return new_cols + new_sys_cols
    else:
        return cols

In [37]:
cols = ["name", "id", "url", "ts", "note"]
_move_sys_col_end(cols, sys_cols=["ts","id", "ts2"])

['name', 'url', 'note', 'id', 'ts']